# Lecture Note: Growth-Optimal Portfolio

## By Albert S. (Pete) Kyle

## BUFN400 Fall 2024---University of Maryland 
 
$\def \sm {} \renewcommand{\sm}{ {\scriptstyle{\text{*}}}}$ 
$\def \mm {} \renewcommand{\mm}{{\scriptsize @}}$
$\def \E {} \renewcommand{\E}{\mathrm{E}}$
$\def \e {} \renewcommand{\e}{\mathrm{e}}$
$\def \drm {} \renewcommand{\drm}{\mathrm{\, d}}$
$\def \var {} \renewcommand{\var}{\mathrm{var}}$
$\def \cov {} \renewcommand{\cov}{\mathrm{cov}}$
$\def \corr {} \renewcommand{\corr}{\mathrm{corr}}$
$\def \stdev {} \renewcommand{\stdev}{\mathrm{stdev}}$
$\def \t {} \renewcommand{\t}{^{\mathsf{T}}}$
$\def \comma {} \renewcommand{\comma}{\, , \,}$
$\def \vec {} \renewcommand{\vec}[1]{\mathbf{#1}}$
$\def \skew {} \renewcommand{\skew}{\mathrm{skew}}$
$\def \kurt {} \renewcommand{\kurt}{\mathrm{kurt}}$
$\def \prob {} \renewcommand{\prob}{\textrm{prob}}$
$\def \midx {} \renewcommand{\midx}{\, \mid \,}$


### Introduction

Geometric Brownian motion has some properties which are often a source of confusion among researchers and practitioners.

This lecture reviews these issues.

### Mean and Median Returns

Consider an investment which follows a **random walk** (idendically and independently returns) with two possible outcomes:

1. With probability 50 percent, the investment doubles in value (1 dollar grows to 2 dollars).

2. With probability 50 percent, the investment halves in value (1 dollar shrinks to 0.50 dollars).

Obviously, the expected gross return is $0.50 * 2.00 + 0.50 * 0.50 = 1.25$, so one dollar is expected to grow to 1.25 dollars. The expected net return is $0.50 * 2.00 + 0.50 * 0.50 - 1.00 = 0.25 = 25 \text{ percent}$.

Although the expected net return is 25 percent, the "effective median" return is 0 percent since doubling and halving converts one dollar into one dollar. More precisely, if the investment is held for an even number of time periods, the median return is exactly 0 percent.

If an investor holds this investment for many periods, approximately half of the sample paths will have an internal rate of return less than zero and approximately half of the sample paths wil have an internal rate of return greater than zero. This occurs because there will more good periods than bad periods about half the time, and vice versa.  There is also the borderline case where the number of good outcomes is exactly the same as the number of bad outcomes.  This is the theoretical median outcome.

Here is the puzzle: Although the expected return is 25 percent per year, half of all sample paths have an actual return less than zero, even if the investment is held for many periods.


### Mathematical explanation for the puzzle

The mathematical explanation for the puzzle is related to the $\sigma^2 / 2$ term that must be subtracted from the mean when simulating a Brownian motion which will be converted into a geometric Brownian by the exponentiation. The convexity of the $\exp$ function makes the median different from the mean (due to Jensen's inequality).

Although the two-outcome example is not geometric Brownian motion, the adjustment is approximately the same as $\sigma^2 / 2$:


In [1]:
import numpy as np

probs = np.array([1.00, 1.00])
probs = probs / probs.sum()

rets = np.array([2.00, 0.50])
arithmetic_mean = probs.dot(rets)
arithmetic_variance = probs.dot(rets**2) - arithmetic_mean**2
arithmetic_variance_adjustment = 0.50 * arithmetic_variance

geometric_mean = (rets**probs).prod()
log_mean = probs.dot(np.log(rets))
log_variance = probs.dot(np.log(rets)**2) - log_mean**2
log_variance_adjustment = 0.50 * log_variance

print(f"{arithmetic_mean=}\n {geometric_mean=}")
print(f"approximation: {arithmetic_variance_adjustment=}\ncorrect: {log_variance_adjustment=}")



arithmetic_mean=1.25
 geometric_mean=1.0000000000000002
approximation: arithmetic_variance_adjustment=0.28125
correct: log_variance_adjustment=0.2402265069591007


### Intuitive explanation for the puzzle

An intuitive explanation for the puzzle is that most of the expected value of returns over multiple periods comes from the right tail of the distribution (i.e., the ones with the largest number of "good" outcomes).

The distribution of compounded returns over many periods is skewed to the right.

Consider what happens if returns are $n$ sample paths are simulated for $m$ periods:



In [3]:
n = 1000
ntop = 10

m = 20

up = 2.00
down = 0.50
prob = 0.50

rng = np.random.default_rng(1234)

#rets = rng.random((n, m))
#rets = up * (rets > prob) + down * (rets <= (1 - prob))

rets = rng.choice([up, down], (n, m), p=[prob, 1.00 - prob])

print(rets[0:2, :])

cum_rets = rets.prod(axis=1)
cum_rets.sort()
cum_rets = cum_rets[::-1]

# print(cum_rets)

# Percent of value from ntop returns:
frac_from_best = cum_rets[:ntop].sum() / cum_rets.sum()

print(f"A fraction {frac_from_best} of returns comes from the top {ntop} of {n} returns.")



[[0.5 2.  0.5 2.  2.  2.  2.  2.  0.5 2.  2.  0.5 0.5 0.5 0.5 0.5 0.5 2.
  2.  0.5]
 [2.  0.5 0.5 0.5 2.  0.5 2.  0.5 2.  2.  0.5 2.  0.5 0.5 2.  0.5 2.  0.5
  2.  2. ]]
A fraction 0.6718453572603227 of returns comes from the top 10 of 1000 returns.


The example in the cell above shows that, when held for 20 periods, over half the expected value comes from the top one percent of returns.

This result has practical implications.  Consider the portfolio of a venture capital fund, which invests in many companies whose value approximately doubles or halves each year. As time passes, most of the value of the investments in the fund becomes concentrated in the small fraction of investments which have performed well.

### Limit of geometric Brownian motion

As the number of periods $m$ becomes very large, almost all of the expected value comes from a very small percentage of the good outcomes.

A related fact: If a geometric Brownian motion has a mean of 1 (i.e., expected return of zero), probablity of obtaining an outcome close to zero (i.e., expected return close to $-100$ percent) goes to one as the number of periods goes to infinity.

### Problem 1

1. Play with the parameter values in the example to verify that value becomes concentrated in the good tail of the distribution.

2. Verify that the outcomes of geometric Brownian motion become small (relative to the mean) with high probability when the variance becomes large.

### Practical implications

These simple facts about random walks or geometric Brownian motion have numerous practical implications.

1.  Some people say, "Due to the way in which venture capitalists invest, most of the returns from a venture capital fund come from a few good investments." This statement is a simple consequence of the properties of geometric Brownian motion. Venture capital investments have very high variance, sometimes approximately 100 percent per year. Returns also have a shape not too different from a lognormal.  Therefore the statement is simply a property of the high variance of the approximately lognormal investment.  It depends on nothing else which is specific to venture capital.

2. Some people say, "Hedge funds offer risky returns, but most of them underperform their benchmark over time. Therefore, hedge funds are bad investments." This statement may be incorrect or misleading in several respects. First, most hedge funds have relatively low volatilities of 10-15 percent per year. Second, if hedge funds did have high volatility and returns approximated geometric Brownian motion, then most hedge funds would underperform over time (by half their variance, i.e., 50 basis points per year for a hedge fund with 10 percent annual volatility), even if the hedge funds did match or exceed their benchmarks on average.

3. Some people say, "Long term internal rate of return is a good statistic for estimating long-term mean returns".  This statement is false because the long-term IRR measures the mean of the log returns, not mean returns.  It is lower than the actual mean by half the variance if returns follow geometric Brownian motion.

4. Some people say, "If a portfolio has good mean returns and you invest with more leverage (e.g., 60 percent, 100 percent, 200 percent, exposure to the asset), then your investment will have better performance in the long run."  This statement is false.  We will explain why next.


### Internal rate of return 

Consider a log random-walk with gross returns $R = (R_1, R_2, ..., R_T)$, and cumulative returns $(P = P_1, P_2, ..., P_T)$ obtained from the cumulative product of gross returns $P_t = R_1 \sm R_2 \sm \sm \sm R_t$. Assume the gross returns $R_t$ are lognormally distributed random variables of the form $R_t = \e^{r_t}$, where $r_t$ is distributed $\mathcal{N}(\bar{r}, \sigma^2)$. The mean return is $\bar{R} = \e^{\bar{r} + \sigma^2 / 2}$.

The **gross internal rate of return**, which is defined as $P_T^{1/T}$, is the **geometric mean** of the individual gross returns $R_t$:

\begin{align}
P_T^{1/T} &= \exp \left( \frac{1}{T} \sm \log P_T \right) \\
&= \exp  \left( \frac{1}{T} \sm \sum_t \log R_t \right) \\
&= \exp  \left( \frac{1}{T} \sm \sum_t r_t \right). 
\end{align}

If one dollar is invested with a constant rate of return equal to the gross internal rate of return, it grows to the same value $P_T$ over $T$ periods as the actual risky investment. The gross internal rate of return is a random variable which depends on the actual random gross returns $R = (R_1, R_2, ..., R_T)$.

The **net (log) internal rate of return** is the log of the gross internal rate of return:

\begin{align}
\log \left( P_T^{1/T} \right) &= \frac{1}{T} \sm \sum_t r_t . 
\end{align}

Thus, the (log) net internal rate of return is the sample mean of the log returns $r_t$.

Now consider the difference between the expectation of the log of the internal rate of return and the log of the expected internal rate of return.

The log of the expected internal rate of return is

\begin{align}
\log \left( \E \left[ P_T^{1/T} \right) \right] 
&= \log \left( \E \left[ \exp  \left( \frac{1}{T} \sm \sum_t r_t \right) \right] \right). 
\end{align}

Now $\frac{1}{T} \sm \sum_t r_t$ is distributed $\mathcal{N}(\bar{r}, \sigma^2 / T)$, so $\E \left[ \exp  \left( \frac{1}{T} \sm \sum_t r_t \right) \right] = \exp ( \bar{r} + \tfrac{1}{2} \sm \sigma^2 / T )$ .

Therefore, the log of the expected internal rate of return is

\begin{align}
\log \left( \E \left[ P_T^{1/T} \right) \right] 
&= \log \left( \exp ( \bar{r} + \tfrac{1}{2} \sm \sigma^2 / T ) \right) \\
&= \bar{r} + \tfrac{1}{2} \sm \sigma^2 / T \\ 
\end{align}

The log of the expected internal rate of return is the log of the expected gross internal rate of return.

### Problem 2

Explain with more detail why the investor with a short position trades in a destabilizing manner.

### Dynamic Trading Strategies

Now suppose that in investor invests a fraction $x$ of wealth in the risky asset and fraction $1-x$ in a safe asset with gross return $R_f$. Suppose the annualized expected gross return of the risky asset is expressed as $R_f + \pi$, and the annualized log variance is approximately $\sigma^2$. The continuously compounded expected return of the portfolio is $r_f + x \sm \pi$ (and the expected return, not compounded, is $R_f\sm \mathrm{e}^\pi$), and the log-variance is $x^2 \sm \sigma^2$.

Now there are some theoretical and practical issues concerned with implementing such an investment strategy over time. 

1. If $ x = 0$, the investor fully invested in the risk-free asset and does not need to trade.

2. If $x = 1$, the investor is fully invested in the risky asset and does not need to trade.

3. if $0 < x < 1$, the investor needs to rebalance the portfolio over time.  At the end of each period, the investor holds a fraction greater than $x$ in the risky asset if its return was greater than the risk-free asset and holds a fraction less than $x$ is its return was less than the risk-free asset.  Therefore, to maintain the target fraction $x$, the investor needs to engage in a dynamic trading strategy of selling the risky asset when its price rises and buying the risky asset when its price falls. This strategy is **stabilizing** in the sense that its price impact goes against market movement.

4. If $x > 1$, the investor borrows at the risk-free rate to invest more than 100 percent of wealth in the risky asset. We can think of the borrowing as taking place in a margin account. Now the investor still needs to rebalance the portfolio over time, but the rebalancing strategy becomes destabilizing.  At the end of each period, the investor holds a fraction less than $x$ in the risky asset if its return was greater than the risk-free asset and holds a fraction more than $x$ is its return was less than the risk-free asset.  Therefore, to maintain the target fraction $x$, the investor needs to engage in a dynamic trading strategy of buying the risky asset when its price rises and selling the risky asset when its price falls. This strategy is **destabilizing** in the sense that its price impact amplifies market movement.

5. If $x < 0$, the investor borrows the asset to sell it short and has more than 100 percent of wealth ($1 - x$, with $x$ negative)at the risk-free rate. Now the investor still needs to rebalance the portfolio over time, but the rebalancing strategy remains destabilizing, like the leveraged long investor with $x > 1$.  Therefore, to maintain the target fraction $x$, the investor needs to engage in a dynamic trading strategy of buying the risky asset when its price rises and selling the risky asset when its price falls. This strategy is **destabilizing** in the sense that its price impact amplifies market movement.

If the the time interval representated by period is very short, then the investor can be viewed as almost rebalancing continuously.  If the time interval is long, the the investor rebalances infrequently. Theoretically, this allows bad things to happen if the investor is leveraged long ($x > 1$) or short ($x < 0$). 

1. If the price falls more than 50 percent (relative to the risk-free asset) in one period, then a leveraged long investor with $x > 2$ is wiped out; he should have sold as prices fell to meet margin calls but did not do so.  

2. If the price rises more than 100 percent (relative to the risk-free asset) in one period, the an short investor with $x < -1$ is also wiped out; he should have bought back his short position as prices fell but did not do so.

In practical market conditions, investors do not rebalance continuously to avoide trading costs, including temporary market impact from buying or selling large amount quickly.  Large investors therefore smooth their trading out over time. If the smoothing takes place over a time period which is too long, then the investor needs to pay attention to his ability to meet margin calls.

### Practical  Examples

There are examples of temporary market dislocation are related to the speed with which large investors rebalance portfolios.

1. In the 1987 stock market crash, portfolio insurers had changing target portfolios which involved selling when prices fell. When prices did fall, their selling seems to have exacerbated the collapse in prices and amplified the market crash.  I think it converted a decline of a 2--3 percent into a decline of 30 percent.

2. The collapse of Archegos Capital may be related to temporary price impact making it impossible to liquidate gigantice positions quickly enough to avoid collapse.

### Growth optimal portfolio

Now let us suppose that the time period between rebalancings and the temporary market impact of rebalancing are both small enough that the investor can be approximately assumed to trade continuously. Over a long period of time, the asset approximately follows geometric Brownian. Recall that the expected return of the portfolio is $R_f + x \sm \pi$, and the variance is $x^2 \sm \sigma^2$. The expected log return (long-run IRR) is $R_f + x \sm \pi - \tfrac{1}{2} \sm x^2 \sm \sigma^2$.

Now suppose that the investor has the objective of achieving the highest possible internal rate of return. The investor choose $x$ to maximize

$$
R_f + x \sm \pi - \tfrac{1}{2} \sm x^2 \sm \sigma^2.
$$

Obviously, the first-order condition for this optimization problem is

$$
x \sm \sigma^2 = \pi
$$

This implies the demand function

$$
x = \frac{\pi}{\sigma^2}.
$$

Intuitively, the investor chooses $x$ to balance a tradeoff between risk $\sigma^2$ and return $\pi$. 

Intitively, the investor also chooses between sacrificing expected return if $x$ is too small while avoiding "blowing up" if $x$ is too large. Investors who are too highly leveraged will suffer gigantic drawdowns which make their achieved IRR low, even if their expected return (overwhelmingly coming from a small number "best" paths) is high.

### Sharpe ratio and volatility

The Sharpe ratio is defined as $\pi / \sigma$.  We can write the optimal demand as

$$
x \sm \sigma = \frac{\pi}{\sigma}.
$$

In this equation, the left side is the volatility of the portfolio.  The right side is the Sharpe ratio. Our result says that an investor maximizing long-term IRR chooses portfolio volatility to equal the Sharpe ratio.



### Practical considerations

In practice, the investor's estimate of mean and variance will change over time. If so, the investor maximizing long-run IRR continues to engage in the short-run growth optimal strategy, choosing portfolio volatility to equal the Sharpe ratio, while adjusting the portfolio volatility over time.

This is a surprizing result which confuses investors, researchers, and politicians. Many people have the intuition that financial investors ignore long-run returns when they focus on short-term returns. The dynamic growth optimal portfolio result shows that this intuition is wrong. An investor who maximizes long-run IRR does indeed focus on the short-term risk-return tradeoff.

### Risk aversion

A simplistic, intuitive way to generalize the demand function is to introduce a **risk aversion parameter** $\rho$, then write the demand function as

$$
x = \frac{\pi}{\rho \sm \sigma^2}.
$$

If $\rho = 1$, the solution is the growth-optimal portfolio. Finance researchers call this solution the solution for a **log utility** investor. If $\rho > 1$, the investor is **more risk averse than log** and holds a more conservative portfolio.  If $\rho > 1$, the investor is **less risk averse than log** and holds a portfolio riskier than the growth-optimal portfolio.

There is a large finance research literature which deals with this problem.  The above equation can be thought of as a reasonable approximation to the solution, not the exact solution.

One issue concerns what happens when $\pi$ is negative and the investor wants to go short.  Mathematically, the demand says the standard deviation is negative.  While this does not make sense mathematically, it does capture correctly the finance intuition that the investor has negative exposure to the risky asset.  It can be shown that what really matters is the **squared Sharpe ratio**.

`

### Notes for future revisions

1. Random Walk

2. Constant investment opportunity set.

3. Log utility

4. Growth-optimal portfolio

5. Bootstrapping